In [1]:
from dask.distributed import Client

client = Client(n_workers=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50471,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:50486,Total threads: 2
Dashboard: http://127.0.0.1:50487/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:50474,


In [2]:
import dask.dataframe as dd
fp = "./incident_event_log.csv"
ddf = dd.read_csv(fp)
ddf

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,bool,int64,int64,int64,bool,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,bool,bool,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
ddf_c = ddf.loc[ddf["incident_state"] == "Resolved"].compute().copy()

In [4]:
del ddf

In [5]:
ddf_c

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
11,INC0000047,Resolved,True,1,0,7,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00
18,INC0000057,Resolved,True,0,0,5,True,Caller 4416,Opened by 8,29/2/2016 06:10,...,False,Do Not Notify,Problem ID 2,?,?,?,code 10,Resolved by 5,1/3/2016 02:55,6/3/2016 03:00
22,INC0000060,Resolved,True,0,0,2,True,Caller 4491,Opened by 180,29/2/2016 06:38,...,False,Do Not Notify,?,?,?,?,code 3,Resolved by 113,2/3/2016 12:06,7/3/2016 13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141668,INC0118999,Resolved,True,0,0,5,True,Caller 2058,Opened by 194,10/2/2017 15:55,...,True,Do Not Notify,?,?,?,?,code 6,Resolved by 110,10/2/2017 16:35,15/2/2017 17:00
141679,INC0119390,Resolved,True,3,0,5,True,Caller 5100,Opened by 490,13/2/2017 10:33,...,True,Do Not Notify,?,?,?,?,code 6,Resolved by 166,14/2/2017 09:37,14/2/2017 12:08
141694,INC0120304,Resolved,True,0,0,1,True,Caller 90,Opened by 8,15/2/2017 02:02,...,True,Do Not Notify,?,?,?,?,code 6,Resolved by 5,17/2/2017 00:47,17/2/2017 00:50
141706,INC0120835,Resolved,True,1,0,3,True,Caller 116,Opened by 12,16/2/2017 09:09,...,True,Do Not Notify,?,?,?,?,code 9,Resolved by 9,16/2/2017 09:53,16/2/2017 09:53


In [6]:
TIMESTAMP_COLS = ["opened_at", "resolved_at"]
ACTIVITY_COLS = ["opened_by", "resolved_by"]
COLS_NEEDED = ACTIVITY_COLS + TIMESTAMP_COLS

In [7]:
ddf_c = ddf_c[COLS_NEEDED]

In [8]:
df_complete = ddf_c[~((ddf_c["opened_at"] == "?") | (ddf_c["resolved_by"] == "?")\
                     | (ddf_c["resolved_at"] == "?") | (ddf_c["opened_by"] == "?"))].copy()

In [9]:
del ddf_c

In [10]:
df_complete

,opened_by,resolved_by,opened_at,resolved_at
1,Opened by 8,Resolved by 149,29/2/2016 01:16,29/2/2016 11:29
2,Opened by 8,Resolved by 149,29/2/2016 01:16,29/2/2016 11:29
11,Opened by 397,Resolved by 81,29/2/2016 04:40,1/3/2016 09:52
18,Opened by 8,Resolved by 5,29/2/2016 06:10,1/3/2016 02:55
22,Opened by 180,Resolved by 113,29/2/2016 06:38,2/3/2016 12:06
...,...,...,...,...
141668,Opened by 194,Resolved by 110,10/2/2017 15:55,10/2/2017 16:35
141679,Opened by 490,Resolved by 166,13/2/2017 10:33,14/2/2017 09:37
141694,Opened by 8,Resolved by 5,15/2/2017 02:02,17/2/2017 00:47
141706,Opened by 12,Resolved by 9,16/2/2017 09:09,16/2/2017 09:53


In [11]:
df_complete["opened_by"] = df_complete["opened_by"].astype(str)
df_complete["resolved_by"] = df_complete["opened_by"].astype(str)
df_complete["opened_at"] = df_complete["opened_at"].astype("datetime64[ns]")
df_complete["resolved_at"] =  df_complete["resolved_at"].astype("datetime64[ns]")

In [12]:
df_complete.dtypes

opened_by              object
resolved_by            object
opened_at      datetime64[ns]
resolved_at    datetime64[ns]
dtype: object

In [13]:
df_Q1 = df_complete[df_complete["resolved_at"].dt.quarter == 1]

In [14]:
del df_complete

In [15]:
df_Q1

,opened_by,resolved_by,opened_at,resolved_at
1,Opened by 8,Opened by 8,2016-02-29 01:16:00,2016-02-29 11:29:00
2,Opened by 8,Opened by 8,2016-02-29 01:16:00,2016-02-29 11:29:00
11,Opened by 397,Opened by 397,2016-02-29 04:40:00,2016-01-03 09:52:00
18,Opened by 8,Opened by 8,2016-02-29 06:10:00,2016-01-03 02:55:00
22,Opened by 180,Opened by 180,2016-02-29 06:38:00,2016-02-03 12:06:00
...,...,...,...,...
141606,Opened by 162,Opened by 162,2017-08-02 14:58:00,2017-02-13 14:42:00
141679,Opened by 490,Opened by 490,2017-02-13 10:33:00,2017-02-14 09:37:00
141694,Opened by 8,Opened by 8,2017-02-15 02:02:00,2017-02-17 00:47:00
141706,Opened by 12,Opened by 12,2017-02-16 09:09:00,2017-02-16 09:53:00
